# PRAW e PSAW - Python + Reddit

* **PRAW** - Módulo que usa a API oficial do **Reddit**.
* **PSAW** - Módulo que usa a API oficial do **Pushshift.io** (ferramenta de busca).

* Pegar o client id e o secret id em https://www.reddit.com/prefs/apps
* Para obter e usar um Refresh Token (**opcional**, difícil): https://praw.readthedocs.io/en/latest/tutorials/refresh_token.html

# Objetivo: analisar posição dos mods do r/brasil sobre desabafos, das últimas 9 semanas

**Ferramenta utilizada: Python Reddit API Wrapper (PRAW)**
* Documentação do PRAW: https://praw.readthedocs.io/en/latest/
  
Para usar a ferramenta, é preciso usar suas credenciais do reddit e seguir um passo a passo descrito na documentação.

A ferramenta não considera posts/comentários removidos pelos usuários e moderadores.

Removi deste post o código no qual inicializo a API do Reddit usando o PRAW, por envolver minhas credenciais.

## Exemplo de como a API do Reddit funciona:

In [77]:
print('\nTeste: 3 posts mais recentes do r/brasil\n')
for submission in reddit.subreddit("brasil").new(limit=3):
    print(submission.title)     #imprime o título do post
    print(submission.url)       #imprime o link do post (ou o permalink, se for selfpost)
    print(submission.permalink) #imprime o permalink do post
    print()                     #imprime uma linha em branco


Teste: 3 posts mais recentes do r/brasil

Como Colocar Crédito pela internet para outra pessoa?
https://www.reddit.com/r/brasil/comments/milvh7/como_colocar_crédito_pela_internet_para_outra/
/r/brasil/comments/milvh7/como_colocar_crédito_pela_internet_para_outra/

A Orcrim-FB: Flávio Bolsonaro e o roubo de 6,1 milhões de reais de dinheiro público
https://piaui.folha.uol.com.br/materia/a-orcrim-fb/
/r/brasil/comments/milr5t/a_orcrimfb_flávio_bolsonaro_e_o_roubo_de_61/

Na live do Osama, Bolsonaro faz discurso terrorista e repete "meu Exército"
https://noticias.uol.com.br/colunas/reinaldo-azevedo/2021/04/02/na-live-do-osama-bolsonaro-faz-discurso-terrorista-e-repete-meu-exercito.htm
/r/brasil/comments/milm74/na_live_do_osama_bolsonaro_faz_discurso/



# Importando bibliotecas e inicializando variáveis

In [46]:
from collections import Counter   #para fazer contagens
import arrow                      #para conferir data do post
import webbrowser                 #usei para abrir posts no meu navegador

In [20]:
# lista de moderadores, obtida em https://www.reddit.com/r/brasil/about/moderators

mods=['bgh251f2',
'botbr',
'Donnutz',
'Gustavobc',
'psychoboxing',
'Caos2',
'rubensheik',
'vintagedan',
'platypup',
'1Pesadelo',
'Santinh0',
'criatura',
'modsbr',
'Usuario-desconhecido',
'AutoModerator',
'JorgeAmVF',
'Utumno_visitor',
'Tiranossauro_Red',
'Tetizeraz',
'Chaos--AD',
'gschizas']

In [84]:
now=arrow.now() #variável que representa a data de hoje

In [87]:
#definindo função auxiliar para printar url dos posts/comentários
def print_url(permalink):
    print(f'https://reddit.com{permalink}')

# Obtendo posts dos mods, no r/brasil, nas últimas 9 semanas:

In [88]:
flairs=[] #para salvar as flairs de todos os posts salvos
posts_por_mod={} #para salvar todos os posts de cada moderador

for mod in mods: #para cada mod na lista de mods
    print(f'\n*** MOD: {mod} ***\n')
    subs=reddit.redditor(mod).submissions.new(limit=None) #pega todos os posts mais recentes do mod
    temp=[]
    for sub in subs:
        diff=now-arrow.get(sub.created_utc)
        if diff.days>63: 
            break #para de pegar posts do mod atual se o post tiver sido feito há mais de 9 semanas
        if sub.subreddit=='brasil':
            flairs.append(sub.link_flair_text) #salva e printa posts dos mods no r/brasil
            temp.append(sub)
            print(sub.title)
            print_url(sub.permalink)
    posts_por_mod[mod]=temp
print('\nTerminado.')


*** MOD: bgh251f2 ***


*** MOD: botbr ***


*** MOD: Donnutz ***

Nova lei exigindo presidentes somente com experiencia
https://reddit.com/r/brasil/comments/lhiyy6/nova_lei_exigindo_presidentes_somente_com/

*** MOD: Gustavobc ***


*** MOD: psychoboxing ***


*** MOD: Caos2 ***

Violência obstétrica atinge quase metade das mães no SUS, mas é normalizada - 30/03/2021 - Equilíbrio e Saúde - Folha
https://reddit.com/r/brasil/comments/mh1gzg/violência_obstétrica_atinge_quase_metade_das_mães/
Villas Bôas tenta apagar história ao negar sequestro de crianças na ditadura - 30/03/2021 - Ilustríssima - Folha
https://reddit.com/r/brasil/comments/mh1fu4/villas_bôas_tenta_apagar_história_ao_negar/
Funcionários da Saúde de SP, incluindo jovens, têm prioridade em vacinação contra Covid - 27/03/2021 - Equilíbrio e Saúde - Folha
https://reddit.com/r/brasil/comments/mf0m6q/funcionários_da_saúde_de_sp_incluindo_jovens_têm/
Grupo de mais de 300 diplomatas publica carta para pedir saída de Ernesto - 27/

## Contando as flairs dos posts dos mods

Fiz isso pra ver quantos posts tinham a flair 'META', normalmente usada pelos mods pra fazer anúncios relacionados ao sub

In [89]:
Counter(flairs).most_common()

[(None, 43),
 ('Notícia', 22),
 ('META', 12),
 ('Artigo', 8),
 ('Entretenimento e Cultura', 5),
 ('Ei, r/brasil', 4),
 ('Discussão', 2),
 ('COVID-19', 2),
 ('Vídeo', 1),
 ('ENEM', 1),
 ('Política', 1),
 ('Humor', 1),
 ('Ciência & Tecnologia', 1),
 ('Cultural Exchange', 1)]

Achei estranho que tem muito post de mod sem flair (indicado por *None*). Decidi pesquisar mais a fundo:

In [90]:
lista=[]
for lista_posts in posts_por_mod.values():
    for post in lista_posts:
        if post.link_flair_text==None:
            print(post.title)
            print(post.url) #postando o link ao invés do permalink do post
            lista.append(post.author) #salva numa lista o autor de cada post sem flair

Violência obstétrica atinge quase metade das mães no SUS, mas é normalizada - 30/03/2021 - Equilíbrio e Saúde - Folha
https://www1.folha.uol.com.br/equilibrioesaude/2021/03/violencia-obstetrica-atinge-quase-metade-das-maes-no-sus-mas-e-normalizada.shtml
Villas Bôas tenta apagar história ao negar sequestro de crianças na ditadura - 30/03/2021 - Ilustríssima - Folha
https://www1.folha.uol.com.br/ilustrissima/2021/03/villas-boas-tenta-apagar-historia-ao-negar-sequestro-de-criancas-na-ditadura.shtml
Funcionários da Saúde de SP, incluindo jovens, têm prioridade em vacinação contra Covid - 27/03/2021 - Equilíbrio e Saúde - Folha
https://www1.folha.uol.com.br/equilibrioesaude/2021/03/funcionarios-da-saude-de-sp-incluindo-jovens-tem-prioridade-em-vacinacao-contra-covid.shtml
Grupo de mais de 300 diplomatas publica carta para pedir saída de Ernesto - 27/03/2021 - Mundo - Folha
https://www1.folha.uol.com.br/mundo/2021/03/grupo-de-mais-de-300-diplomatas-publica-carta-para-pedir-saida-de-ernesto.s

In [91]:
Counter(lista)

Counter({Redditor(name='Caos2'): 42, Redditor(name='modsbr'): 1})

Parece que a maioria dos posts são de um só mod, e são só sobre notícias.

## Buscando posts com flair 'META'

In [92]:
for lista_posts in posts_por_mod.values():
    for post in lista_posts:
        if post.link_flair_text=='META':
            print(post.title)
            print(post.url) 

MEGATHREAD - Mudanças nos ministérios e outros cargos políticos, 23/03/2021
https://www.reddit.com/r/brasil/comments/mfymh4/megathread_mudanças_nos_ministérios_e_outros/
Mudança no texto sobre flairs na nossa Wiki e novo texto na nossa wiki.
https://www.reddit.com/r/brasil/comments/md58xi/mudança_no_texto_sobre_flairs_na_nossa_wiki_e/
Possível AMA/PQC com Júlio Lancellotti, e uma nota de repúdio da moderação quanto à uma fala de Bia Dória.
https://www.reddit.com/r/brasil/comments/m8v52a/possível_amapqc_com_júlio_lancellotti_e_uma_nota/
Posts de desabafos agora vão para o r/desabafos, e um aviso sobre o uso de Flairs incorretamente
https://www.reddit.com/r/brasil/comments/l7xnea/posts_de_desabafos_agora_vão_para_o_rdesabafos_e/
Bate-papo de Sexta - Que podcasts você está ouvindo? - 02.04.2021
https://www.reddit.com/r/brasil/comments/mim3at/batepapo_de_sexta_que_podcasts_você_está_ouvindo/
AMA/PQC amanhã, Quarta-feira 31/03, com um "nômade digital"
https://www.reddit.com/r/brasil/comment

**O único post dos mods sobre desabafos nas últimas 9 semanas é o post anunciando que eles seriam removidos.**

Só pra ter certeza, vou procurar por posts dos mods sobre desabafos, independente da flair.

## Buscando posts dos mods, no r/brasil, nas últimas 9 semanas, com o termo "desabafo"

In [93]:
for lista in posts_por_mod.values():
    for post in lista:
        if 'desabafo' in post.selftext.lower() or 'desabafo' in post.title.lower():
            print(post.title)
            print_url(post.permalink)

Posts de desabafos agora vão para o r/desabafos, e um aviso sobre o uso de Flairs incorretamente
https://reddit.com/r/brasil/comments/l7xnea/posts_de_desabafos_agora_vão_para_o_rdesabafos_e/
Convido vocês para o r/conversas!
https://reddit.com/r/brasil/comments/lnjdhe/convido_vocês_para_o_rconversas/


**O único post dos mods sobre desabafos, no r/brasil, nas últimas 9 semanas, é o post dizendo que os desabafos seriam removidos por um mês.**

# Obtendo comentários dos mods, no r/brasil, nas últimas 9 semanas, contendo o termo "desabafo"

In [94]:
coms_por_mod={}

for mod in mods:
    print(f'\n*** MOD: {mod} ***\n')
    coms=reddit.redditor(mod).comments.new(limit=None)
    temp=[]
    for com in coms:
        diff=now-arrow.get(com.created_utc)
        if diff.days>63:
            break
        if 'desabafo' in com.body.lower() and com.subreddit=='brasil': #salva comentários dos mods com "desabafo"
            temp.append(com)
            print(com.body)
            print_url(com.permalink)
            print()
    coms_por_mod[mod]=temp
print('\nTerminado.')


*** MOD: bgh251f2 ***

Bem, se as pessoas começarem a postar mais coisas variadas a gente pode pensar em diversificar e dividir mesmo algumas delas. Mas também já aconteceu da gente retirar algumas flairs para podermos direcionar as postagens em específico para subs mais qualificados como o /r/desabafos.
https://reddit.com/r/brasil/comments/md58xi/mudança_no_texto_sobre_flairs_na_nossa_wiki_e/gsa1uzj/


*** MOD: botbr ***


*** MOD: Donnutz ***


*** MOD: Gustavobc ***


*** MOD: psychoboxing ***


*** MOD: Caos2 ***


*** MOD: rubensheik ***


*** MOD: vintagedan ***


*** MOD: platypup ***


*** MOD: 1Pesadelo ***


*** MOD: Santinh0 ***


*** MOD: criatura ***


*** MOD: modsbr ***

Ah, só pra falar, vamos sim remover alguns desabafos que estão na frontpage agora, mas precisamos colocar os removal reasons, etc. Pra notificar a pessoa corretamente. Talvez leve uns minutos.
https://reddit.com/r/brasil/comments/l7xnea/posts_de_desabafos_agora_vão_para_o_rdesabafos_e/gl9ausw/


*** MOD

**Nenhum comentário dos mods fala sobre a volta dos desabafos.**

# F desabafos no r/brasil 😔